In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, precision_score, recall_score, f1_score
import polars as pl
from model import UserBasedCF, ItemBasedCF, MatrixFactorizationCF
import matplotlib.pyplot as plt

In [ ]:
# Function to evaluate the model
def evaluate_model(model, test_data):
    y_true = []
    y_pred = []

    for row in test_data.itertuples():
        user_id = row.user_id
        game_id = row.game_id
        actual_rating = row.rating

        prediction = model.predict(user_id, game_id)
        y_true.append(actual_rating)
        y_pred.append(prediction)

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Mean Absolute Error
    mae = mean_absolute_error(y_true, y_pred)

    # Root Mean Squared Error
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    # Precision, Recall, F1-score (assuming threshold for relevant rating is > 3)
    y_true_binary = (y_true > 3).astype(int)
    y_pred_binary = (y_pred > 3).astype(int)

    precision = precision_score(y_true_binary, y_pred_binary, zero_division=0)
    recall = recall_score(y_true_binary, y_pred_binary, zero_division=0)
    f1 = f1_score(y_true_binary, y_pred_binary, zero_division=0)

    return {'MAE': mae, 'RMSE': rmse, 'Precision': precision, 'Recall': recall, 'F1-score': f1}

In [ ]:
def plot_evaluation_results(results, model_name):
    labels, values = zip(*results.items())
    x = np.arange(len(labels))
    plt.bar(x, values, color='b')
    plt.xticks(x, labels)
    plt.ylabel('Score')
    plt.title(f'{model_name} Evaluation')
    plt.show()

In [ ]:
# Directory paths
parquet_dir = 'data/parquets'
json_path = 'data/game_details/SteamMostPlayed_game_details.json'
selection_file = None

_, test_data = load_data(parquet_dir, json_path, selection_file)

In [ ]:
# Load models
model_dir = 'data/model'

user_based_model = UserBasedCF.load(f'{model_dir}/user_based_cf_model.pkl')
item_based_model = ItemBasedCF.load(f'{model_dir}/item_based_cf_model.pkl')
matrix_factorization_model = MatrixFactorizationCF.load(f'{model_dir}/matrix_factorization_cf_model.pkl')

In [ ]:
# Evaluate models
user_based_results = evaluate_model(user_based_model, test_data)
item_based_results = evaluate_model(item_based_model, test_data)
matrix_factorization_results = evaluate_model(matrix_factorization_model, test_data)

print("User-Based CF Evaluation Results:", user_based_results)
print("Item-Based CF Evaluation Results:", item_based_results)
print("Matrix Factorization CF Evaluation Results:", matrix_factorization_results)

In [ ]:
# Plot results
plot_evaluation_results(user_based_results, 'User-Based CF')
plot_evaluation_results(item_based_results, 'Item-Based CF')
plot_evaluation_results(matrix_factorization_results, 'Matrix Factorization CF')